In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import warnings
import numpy as np
import urllib
import pyodbc
import sqlalchemy as sa
import math
from openpyxl import load_workbook
import os

warnings.filterwarnings("ignore") # tắt cảnh báo

connection_string = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'SERVER=118.69.201.34;'
    'Database=FLC_SHOP_SALES_DATAWAREHOUSE;'
    'UID=ecom_user;'
    'PWD=Ec0m@12345;'
    'Trusted_Connection=no;') # kết nối database 

connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(str(connection_string))}"
engine = sa.create_engine(connection_uri, fast_executemany=True)

# Step 2: Function xử lý thuộc tính tọa độ

In [ ]:
# Tạo một hàm để nối từ vào cột nếu giá trị khác 0
def append_word(value, word):
    if value != 0 and value != 'bus_stop':
        return word + str(value)
    else:
        return value
    
# Tạo một hàm để nối giá trị từ cột thứ 8 trở đi
def combine_values(row):
    columns_to_combine = row[8:]  # Lấy các giá trị từ cột thứ 8 trở đi
    unique_values = set(columns_to_combine)  # Tạo một set chứa các giá trị duy nhất
    non_zero_values = [str(row[i]) for i in range(8, len(row)) if row[i] != 0]  # Lọc các giá trị khác 0 theo thứ tự cột
    if len(non_zero_values) > 1:  # Kiểm tra nếu có ít nhất 2 giá trị khác 0
        if non_zero_values[1:] == [non_zero_values[0]] * (len(non_zero_values) - 1):  # Kiểm tra nếu các giá trị từ cột thứ hai trở đi trùng với giá trị đầu tiên
            return non_zero_values[0]  # Trả về giá trị đầu tiên
        else:
            return f"{non_zero_values[0]}: ({', '.join(non_zero_values[1:])})"  # Nối giá trị đầu tiên và các giá trị từ giá trị thứ hai trở đi với dấu ngoặc
    elif len(non_zero_values) == 1:  # Kiểm tra nếu chỉ có 1 giá trị khác 0
        return non_zero_values[0]  # Trả về giá trị duy nhất
    else:
        return None  # Trả về None nếu không có giá trị khác 0
    
def replace_values(values):
    values.replace("yes", 0, inplace=True)
    values.replace("no", 0, inplace=True)
    values.replace("unclassified", 0, inplace=True)
    values.fillna(0, inplace=True)

def append_values(values, file_path):
    # Đọc tệp Excel hiện có vào một đối tượng Workbook của openpyxl
    book = load_workbook(file_path)
    # Chọn sheet muốn thêm dữ liệu
    sheet_name = 'Sheet1'
    # Lấy sheet từ Workbook
    sheet = book[sheet_name]
    # Xác định dòng tiếp theo để bắt đầu thêm dữ liệu
    existing_data_rows = sheet.max_row
    start_row = existing_data_rows + 1
    # Kiểm tra và thêm các dòng không trùng lặp
    for index, row in values.iterrows():
        duplicate_row = False
        for i in range(1, existing_data_rows + 1):
            duplicate = True
            for col_index, value in enumerate(row):
                if sheet.cell(row=i, column=col_index+1).value != value:
                    duplicate = False
                    break
            if duplicate:
                duplicate_row = True
                break
        if not duplicate_row:
            for col_index, value in enumerate(row):
                sheet.cell(row=start_row+index, column=col_index+1, value=value)
    book.save(file_path) # Lưu tệp Excel

# Step 2: Lấy danh sách file từ thư mục

In [ ]:
folder_path = r'D:\SHOP_LC'  # Đường dẫn tới thư mục
# Lấy danh sách tệp tin trong thư mục
file_list = os.listdir(folder_path)
# Hiển thị các tệp tin
for file_name in file_list:
    print(file_name)

# Step 2: Làm sạch dữ liệu từ file 

In [ ]:
filepath_in = r'D:\SHOP_LC\shop_35_39_details.xlsx'
filepath_out = r'D:\SHOP_LC\shop_35_39_details.xlsx'
file_path = r'D:\SHOP_LC\SHOP_LC_FINAL.xlsx'
getmap = pd.read_excel(filepath_in, dtype = object, sheet_name = 'Sheet1')
# Danh sách các cột cần kiểm tra và thay thế
columns_to_check = ['SHOP_CODE', 'SHOP_NAME','CENTER_CORDINATES','POINTS_AROUND','OSM_ID', 'OSM_NAME', 'OSM_TYPE', 'LOCATION_DISTANCE',
                    'amenity', 'shop', 'office', 'tourism', 'healthcare', 'building', 'landuse',
                    'residential', 'religion', 'cuisine', 'community_centre', 'government',
                    'denomination', 'highway', 'clothes', 'gambling', 'junction', 'leisure',
                    'sport', 'studio', 'support', 'surface']
# Kiểm tra và thay thế các cột không có trong DataFrame bằng 0
for column in columns_to_check:
    if column not in getmap.columns:
        getmap[column] = 0
get_run = getmap[['SHOP_CODE', 'SHOP_NAME','CENTER_CORDINATES','POINTS_AROUND','OSM_ID', 'OSM_NAME', 'OSM_TYPE', 'LOCATION_DISTANCE','amenity','shop','office','tourism','healthcare','building','landuse','residential','religion','cuisine','community_centre','government','denomination','highway','clothes','gambling','junction','leisure','sport','studio','support','surface']] #lấy những cột cần chạy
replace_values(get_run)
get_run.head(2)

# Step 2: Xử lý các thuộc tính của tọa độ từ dữ liệu

In [ ]:
# Cột cần nối từ
column_name_1, word_to_append_1 = 'highway', 'highway_'
column_name_2, word_to_append_2 = 'shop', 'shop_'
column_name_3, word_to_append_3 = 'surface', 'surface:'
# Áp dụng hàm append_word cho cột cần xử lý và lưu kết quả vào cùng cột
get_run[column_name_1] = get_run[column_name_1].apply(lambda x: append_word(x, word_to_append_1))
get_run[column_name_2] = get_run[column_name_2].apply(lambda x: append_word(x, word_to_append_2))
get_run[column_name_3] = get_run[column_name_3].apply(lambda x: append_word(x, word_to_append_3))
# Áp dụng hàm combine_values cho từng dòng trong DataFrame và lưu kết quả vào cột "combined_values"
get_run['COMBINE_VALUES'] = get_run.apply(combine_values, axis=1)
results = get_run [['SHOP_CODE', 'SHOP_NAME','CENTER_CORDINATES','POINTS_AROUND','OSM_ID', 'OSM_NAME', 'OSM_TYPE', 'LOCATION_DISTANCE','COMBINE_VALUES']] #lấy cột trích xuất
print(len(results))
results.head(2)

# Step 2: Trích xuất File đã xử lý thuộc tính

In [ ]:
# results.to_excel(filepath_out,index=False) # xuất file
append_values(results,file_path)
re_check = pd.read_excel(file_path, dtype = object, sheet_name = 'Sheet1')
print(len(re_check))
re_check.head(2)

# Step 3: Lấy dữ liệu tất cả shop ở Long Châu (Đã xử lý shop vệ tinh xung quanh)

In [5]:
shop_flc = pd.read_excel(r'D:\SHOP_LC\SHOP_LC_FINAL.xlsx') #lấy dữ liệu shop long châu
get_test = shop_flc[['SHOP_CODE','SHOP_NAME','CENTER_CORDINATES','POINTS_AROUND','OSM_ID','OSM_NAME','OSM_TYPE','LOCATION_DISTANCE','COMBINE_VALUES']] #lấy những cột cần chạy
filtered_get_test = get_test.dropna(subset=['CENTER_CORDINATES', 'POINTS_AROUND'], how='all') #loại bỏ những shop có giá trị lat, long null
filtered_get_test[['CENTER_CORDINATES', 'POINTS_AROUND']] = filtered_get_test[['CENTER_CORDINATES', 'POINTS_AROUND']].apply(lambda x: x.str.replace(', ', ','))
filtered_get_test.head(10)

,SHOP_CODE,SHOP_NAME,CENTER_CORDINATES,POINTS_AROUND,OSM_ID,OSM_NAME,OSM_TYPE,LOCATION_DISTANCE,COMBINE_VALUES
0,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7357939,106.7181225",4465767591,"Quán Karaoke King, Nguyễn Lương Bằng, Phường T...",node,20.035111,karaoke_box
1,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7362367,106.7181914",5608878214,"PNJ, Đường Tân Mỹ, Phường Tân Phú, Quận 7, Thà...",node,56.856434,shop_jewelry
2,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7360136,106.7176643",149904642,"Đường Số 10, Phường Tân Phú, Quận 7, Thành phố...",way,75.648795,highway_residential: (surface:asphalt)
3,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7348802,106.7181946",721979572,"Nguyễn Lương Bằng, Phường Tân Phú, Quận 7, Thà...",way,94.967736,highway_tertiary: (surface:asphalt)
4,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7362624,106.7191736",188114147,"Chợ Tân Mỹ, Đường Số 8, Phường Tân Phú, Quận 7...",way,112.539594,marketplace
5,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7357806,106.7193775",205957627,"UBND Phường Tân Phú, Đường Số 8, Phường Tân Ph...",way,118.524904,government
6,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7346868,106.7180540",1157248742,"Bệnh viện Quốc tế Gia Khang - GIH, 2, Nguyễn L...",way,118.687706,hospital
7,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7360433,106.7172144",149904645,"Đường Số 12, Phường Tân Phú, Quận 7, Thành phố...",way,123.082590,highway_residential: (surface:asphalt)
8,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7368844,106.7179593",6772605318,"103, Đường Số 1, Phường Tân Phú, Quận 7, Thành...",node,132.547465,NaN
9,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7370254,106.7179500",6772579455,"98, Đường Số 1, Phường Tân Phú, Quận 7, Thành ...",node,147.853053,NaN


# Step 3: Funtion xử lý tọa độ

In [14]:
def invert_lat_long(lat_long):
    lat, long = lat_long.split(",")
    inverted_lat_long = f"{long},{lat}"
    return inverted_lat_long

def combine_lat_long(startpoint, endpoint):
    inverted_startpoint = invert_lat_long(startpoint)
    inverted_endpoint = invert_lat_long(endpoint)
    latlon = f"{inverted_startpoint};{inverted_endpoint}"
    return latlon

# Hàm truy vấn yêu cầu địa chỉ từ OSRM
def get_directions_bike(latlon):
    try:
        url = "https://routing.openstreetmap.de/routed-bike/route/v1/driving/"
        coordinates = latlon
        params = {
            "steps": "true",
            "overview":"false",
            "alternatives":"true"
        }
        params_text = '?overview=false&alternatives=true&steps=true'
        response = requests.get(url + coordinates,params=params)
    #     print(url + coordinates + params_text)
        data = response.json()
        return data
    except requests.exceptions.Timeout:
        return "Kết nối tới routing.openstreetmap.de đã vượt quá thời gian chờ."
    except requests.exceptions.RequestException as e:
        return f"Lỗi kết nối: {e}"

def get_directions_car(latlon):
    try:
        url = "https://routing.openstreetmap.de/routed-car/route/v1/driving/"
        coordinates = latlon
        params = {
            "steps": "true",
            "overview":"false",
            "alternatives":"true"
        }
        params_text = '?overview=false&alternatives=true&steps=true'
        response = requests.get(url + coordinates,params=params)
    #     print(url + coordinates + params_text)
        data = response.json()
        return data
    except requests.exceptions.Timeout:
        return "Kết nối tới routing.openstreetmap.de đã vượt quá thời gian chờ."
    except requests.exceptions.RequestException as e:
        return f"Lỗi kết nối: {e}"
def get_directions_foot(latlon):
    try:
        url = "https://routing.openstreetmap.de/routed-foot/route/v1/driving/"
        coordinates = latlon
        params = {
            "steps": "true",
            "overview":"false",
            "alternatives":"true"
        }
        params_text = '?overview=false&alternatives=true&steps=true'
        response = requests.get(url + coordinates,params=params)
    #     print(url + coordinates + params_text)
        data = response.json()
        return data
    except requests.exceptions.Timeout:
        return "Kết nối tới routing.openstreetmap.de đã vượt quá thời gian chờ."
    except requests.exceptions.RequestException as e:
        return f"Lỗi kết nối: {e}"
# Lấy thông tin chỉ đường từ JSON
def extract_directions(data):
    if data["code"] == "Ok":
        routes = data.get('routes')
        waypoints = data["waypoints"]
        if routes:
            legs = routes[0].get('legs')
            if legs:
                summary = legs[0].get('summary')
                duration = round(legs[0].get('duration'),2)
                distance = round(legs[0].get('distance')/1000,2)
                # Chuyển đổi giây sang giờ, phút, giây
                hours = int(duration // 3600)
                minutes = int((duration % 3600) // 60)
                seconds = int(duration % 60)
                return f"Duration: {hours}h:{minutes}m:{seconds}s", f"Distance: {distance} Km"
#                 steps = legs[0].get('steps')
#                 for step in steps:
#                     name = step.get('name')
#                     step_duration = step.get('duration')
#                     step_distance = step.get('distance')
#                     maneuver = step.get('maneuver')
#                     print(f"Name: {name}")
#                     print(f"Duration: {step_duration}")
#                     print(f"Distance: {step_distance}")
#                     if maneuver:
#                         location = maneuver.get('location')
#                         print(f"Maneuver Location: {location}")
#                         print("--------")

#         for waypoint in waypoints:
#             print("Name:", waypoint["name"])
#             print("Location:", waypoint["location"])
#             print("Distance:", waypoint["distance"])
#             print()

# Step 3: Trích xuất thời gian, khoảng cách giữa 2 tọa độ

In [15]:
SHOP_CODE = []
SHOP_NAME = []
CENTER_CORDINATES = []
POINTS_AROUND = []
OSM_ID = []
OSM_NAME = []
OSM_TYPE = []
LOCATION_DISTANCE = []
COMBINE_VALUES = []
BIKE = []
CAR = []
FOOT = []
for i in tqdm(range(0,200)):
    startpoint = filtered_get_test['CENTER_CORDINATES'][i]
    endpoint = filtered_get_test['POINTS_AROUND'][i]
    latlon = combine_lat_long(startpoint, endpoint)
    directions_data_bike = get_directions_bike(latlon)
    directions_data_car = get_directions_car(latlon)
    directions_data_foot = get_directions_foot(latlon)
    SHOP_CODE.append(filtered_get_test['SHOP_CODE'][i])
    SHOP_NAME.append(filtered_get_test['SHOP_NAME'][i])
    CENTER_CORDINATES.append(filtered_get_test['CENTER_CORDINATES'][i])
    POINTS_AROUND.append(filtered_get_test['POINTS_AROUND'][i])
    OSM_ID.append(filtered_get_test['OSM_ID'][i])
    OSM_NAME.append(filtered_get_test['OSM_NAME'][i])
    OSM_TYPE.append(filtered_get_test['SHOP_CODE'][i])
    LOCATION_DISTANCE.append(filtered_get_test['LOCATION_DISTANCE'][i])
    COMBINE_VALUES.append(filtered_get_test['COMBINE_VALUES'][i])
    BIKE.append(extract_directions(directions_data_bike))
    CAR.append(extract_directions(directions_data_car))
    FOOT.append(extract_directions(directions_data_foot))

100%|██████████| 200/200 [06:48<00:00,  2.04s/it]


# Step 3: Chuyển thành DataFrame

In [10]:
df = pd.DataFrame()
df['SHOP_CODE'] = pd.DataFrame(SHOP_CODE)
df['SHOP_NAME'] = pd.DataFrame(SHOP_NAME)
df['CENTER_CORDINATES'] = pd.DataFrame(CENTER_CORDINATES)
df['POINTS_AROUND'] = pd.DataFrame(POINTS_AROUND)
df['OSM_ID'] = pd.DataFrame(OSM_ID)
df['OSM_NAME'] = pd.DataFrame(OSM_NAME)
df['OSM_TYPE'] = pd.DataFrame(OSM_TYPE)
df['LOCATION_DISTANCE'] = pd.DataFrame(LOCATION_DISTANCE)
df['COMBINE_VALUES'] = pd.DataFrame(COMBINE_VALUES)
df['BIKE'] = pd.Series(BIKE).apply(lambda x: f"{x[0]}, {x[1]}")
df['CAR'] = pd.Series(CAR).apply(lambda x: f"{x[0]}, {x[1]}")
df['FOOT'] = pd.Series(FOOT).apply(lambda x: f"{x[0]}, {x[1]}")
df.head(3)

,SHOP_CODE,SHOP_NAME,CENTER_CORDINATES,POINTS_AROUND,OSM_ID,OSM_NAME,OSM_TYPE,LOCATION_DISTANCE,COMBINE_VALUES,BIKE,CAR,FOOT
0,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7357939,106.7181225",4465767591,"Quán Karaoke King, Nguyễn Lương Bằng, Phường T...",80006,20.035111,karaoke_box,"Duration: 0h:0m:26s, Distance: 0.03 Km","Duration: 0h:0m:15s, Distance: 0.05 Km","Duration: 0h:0m:23s, Distance: 0.03 Km"
1,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7362367,106.7181914",5608878214,"PNJ, Đường Tân Mỹ, Phường Tân Phú, Quận 7, Thà...",80006,56.856434,shop_jewelry,"Duration: 0h:0m:13s, Distance: 0.06 Km","Duration: 0h:0m:4s, Distance: 0.06 Km","Duration: 0h:0m:44s, Distance: 0.06 Km"
2,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7360136,106.7176643",149904642,"Đường Số 10, Phường Tân Phú, Quận 7, Thành phố...",80006,75.648795,highway_residential: (surface:asphalt),"Duration: 0h:0m:28s, Distance: 0.09 Km","Duration: 0h:0m:16s, Distance: 0.09 Km","Duration: 0h:1m:9s, Distance: 0.09 Km"
3,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7348802,106.7181946",721979572,"Nguyễn Lương Bằng, Phường Tân Phú, Quận 7, Thà...",80006,94.967736,highway_tertiary: (surface:asphalt),"Duration: 0h:1m:14s, Distance: 0.16 Km","Duration: 0h:0m:38s, Distance: 0.18 Km","Duration: 0h:1m:15s, Distance: 0.09 Km"
4,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7362624,106.7191736",188114147,"Chợ Tân Mỹ, Đường Số 8, Phường Tân Phú, Quận 7...",80006,112.539594,marketplace,"Duration: 0h:0m:52s, Distance: 0.17 Km","Duration: 0h:0m:28s, Distance: 0.17 Km","Duration: 0h:2m:13s, Distance: 0.17 Km"
5,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7357806,106.7193775",205957627,"UBND Phường Tân Phú, Đường Số 8, Phường Tân Ph...",80006,118.524904,government,"Duration: 0h:0m:41s, Distance: 0.16 Km","Duration: 0h:0m:23s, Distance: 0.16 Km","Duration: 0h:2m:4s, Distance: 0.16 Km"
6,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7346868,106.7180540",1157248742,"Bệnh viện Quốc tế Gia Khang - GIH, 2, Nguyễn L...",80006,118.687706,hospital,"Duration: 0h:0m:56s, Distance: 0.15 Km","Duration: 0h:0m:26s, Distance: 0.18 Km","Duration: 0h:1m:41s, Distance: 0.13 Km"
7,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7360433,106.7172144",149904645,"Đường Số 12, Phường Tân Phú, Quận 7, Thành phố...",80006,123.082590,highway_residential: (surface:asphalt),"Duration: 0h:0m:40s, Distance: 0.14 Km","Duration: 0h:0m:23s, Distance: 0.14 Km","Duration: 0h:1m:49s, Distance: 0.14 Km"
8,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7368844,106.7179593",6772605318,"103, Đường Số 1, Phường Tân Phú, Quận 7, Thành...",80006,132.547465,NaN,"Duration: 0h:0m:48s, Distance: 0.16 Km","Duration: 0h:0m:21s, Distance: 0.16 Km","Duration: 0h:2m:11s, Distance: 0.16 Km"
9,80006,LC HCM 72 Tân Mỹ,"10.735733,106.718295","10.7370254,106.7179500",6772579455,"98, Đường Số 1, Phường Tân Phú, Quận 7, Thành ...",80006,147.853053,NaN,"Duration: 0h:0m:48s, Distance: 0.17 Km","Duration: 0h:0m:22s, Distance: 0.17 Km","Duration: 0h:2m:13s, Distance: 0.17 Km"


In [11]:
df.to_excel(r'D:\SHOP_LC\get_directions.xlsx')